In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import requests_cache

from moonbox import now, get_oneday, parse_oneday, draw_moon

In [ ]:
session = requests_cache.CachedSession("cache")

year = now().year

dates = (
    pl.select(pl.date_range(pl.date(year, 1, 1), pl.date(year, 12, 31)))
    .to_series()
    .to_list()
)

data = [
    {"date": date} | parse_oneday(get_oneday(date.isoformat(), session=session))
    for date in dates
]

In [ ]:
def chunk(lst, pred, min_len=2):
    out = []
    chunk = None
    for elt in lst:
        if chunk is None:
            if pred(elt):
                chunk = [elt]
            else:
                pass
        elif pred(elt) and len(chunk) + 1 > min_len:
            out.append(chunk)
            chunk = [elt]
        else:
            chunk.append(elt)

    return out


# group into lunar months
lunar_months = chunk(data, lambda x: x["phase"] == "New Moon")

n_months = len(lunar_months)
longest_month = max([len(x) for x in lunar_months])

In [ ]:
plt.ioff()

canvas_size = (30.0, 20.0)
moon_radius = 0.4
calendar_x = 1.0
calendar_y = 1.0
spacer_x = 0.1
spacer_y = 0.4

fig = plt.figure(figsize=canvas_size)
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xlim(0, canvas_size[0])
ax.set_ylim(0, canvas_size[1])

ax.set_facecolor("black")

solar_month = None

for row, month in enumerate(lunar_months):
    for col, datum in enumerate(month):
        if col == 0 or datum["date"].month != solar_month:
            label = datum["date"].strftime("%b %-d")
            solar_month = datum["date"].month
        else:
            label = datum["date"].strftime("%-d")

        f = datum["illumination"] / 100
        direction = datum["phase"].split()[0].lower()

        moon_x = calendar_x + (2 * moon_radius + spacer_x) * col
        moon_y = calendar_y + (2 * moon_radius + spacer_y) * row

        draw_moon(
            ax,
            x=moon_x,
            y=moon_y,
            radius=moon_radius,
            f=f,
            direction=direction,
            dark="0.2",
        )
        ax.text(
            x=moon_x,  # x position of end of text
            y=moon_y + moon_radius,  # y position of top of text
            s=label,
            horizontalalignment="right",
            verticalalignment="bottom",
            color="white",
        )

# title
ax.text(
    x=15.0,
    y=20.0,
    s=f"{year} Lunar Months",
    verticalalignment="top",
    horizontalalignment="center",
    fontsize=60,
    color="white",
)

# signature
ax.text(
    x=canvas_size[0],
    y=0.0,
    s="Scott Olesen",
    verticalalignment="bottom",
    horizontalalignment="right",
    fontsize=30,
    color="white",
)

# do not show the figure; rendering in VSCode is very slow
fig.savefig("../output/tmp.png", dpi=72)

In [ ]:
# production quality
fig.savefig("../output/lunar_calendar.png", dpi=300)
fig.savefig("../output/lunar_calendar.pdf")